In [3]:
# import necessary materials
import sqlite3
import pandas as pd
import glob
import os
from tqdm import tqdm
import datetime
from datetime import timedelta


conn = sqlite3.connect(r"D:\NMT\OneDrive\Viettravel Airline\Database\VTA_RM_test.db")
c = conn.cursor()

pd.options.mode.chained_assignment = None

def file_name_modified(path:str):
    all_dir_files = glob.glob(path + '/*')
    m_time = []

    for dir_file in all_dir_files:
        # file modification timestamp of a file
        f_m_time = os.path.getmtime(dir_file)
        # convert timestamp into DateTime object
        f_dt_m = datetime.datetime.fromtimestamp(f_m_time)
        m_time += [f_dt_m]

    files_name = pd.Series(all_dir_files).str.split('\\', expand=True).iloc[:, -1]
    table_name = pd.Series(all_dir_files).str.split('\\', expand=True).iloc[:, -2]
    return pd.DataFrame({'file_name':files_name, 'modified_time':m_time,
                         'table_name':table_name, 'dir_file':all_dir_files})

# create connection to sqlite

cargo = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\cargo'
flown_aircraft_leg = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\flown_aircraft_leg'
inflow_cash = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\inflow_cash'
pax_revenue = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\pax_revenue'
payment_detail = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\payment_detail'
reservation = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\reservation'
transaction = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\pax_transaction'
exchange_rate = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\exchange_rate'

dim_agent = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_agent'
dim_calendar = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_calendar'
dim_fare_code = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_fare_code'
dim_routes = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_routes'
dim_slot_time = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_slot_time'
flight_type = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\flight_type'


dir_list = [cargo, flown_aircraft_leg, inflow_cash, pax_revenue, payment_detail, reservation, exchange_rate, 
        dim_agent, dim_calendar, dim_routes, dim_fare_code, dim_slot_time, flight_type, transaction]
frame_list = []

for dir_x in dir_list:
    frame_list.append(file_name_modified(dir_x))

new_log_table = pd.concat(frame_list, axis=0, ignore_index=True)

query = """
SELECT *
    FROM log_table
"""

previous_log_table = pd.read_sql_query(query, conn)
previous_log_table['modified_time'] = pd.to_datetime(previous_log_table['modified_time'])

diff_table = pd.concat([previous_log_table, new_log_table]).drop_duplicates(ignore_index=True, keep=False)
keep_table = pd.concat([previous_log_table, diff_table, diff_table]).drop_duplicates(ignore_index=True, keep=False)
remove_table = pd.concat([previous_log_table, keep_table]).drop_duplicates(ignore_index=True, keep=False)
add_table = pd.concat([new_log_table, keep_table]).drop_duplicates(ignore_index=True, keep=False)

In [18]:
from pandas.tseries.offsets import MonthEnd

In [59]:
exrate = pd.read_excel(r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\exchange_rate\exchange_rate.xlsx')
exrate['Buy_in_cash'].replace('-', 0, inplace=True)

In [62]:
date_index = pd.date_range(start=exrate['Valid_date'].min(), end=exrate['Valid_date'].max() + MonthEnd(0))
curr_code = exrate['Code'].unique()
new_index = pd.MultiIndex.from_product([date_index, curr_code], names=['Valid_date', 'Code'])
exrate_new = exrate.set_index(['Valid_date', 'Code'])
exrate_new = exrate_new.reindex(new_index).reset_index().sort_values(by=['Code', 'Valid_date']).fillna(method='ffill').reset_index(drop=True)
# exrate_new.to_excel('test.xlsx', index=False)

In [61]:
exrate_new

,Valid_date,Code,Full_name,Buy_in_cash,Buy_in_transfer,Sell
0,2022-10-31,AUD,AUSTRALIAN DOLLAR,15515.27,15671.99,16175.35
1,2022-11-01,AUD,AUSTRALIAN DOLLAR,15497.32,15653.86,16156.63
2,2022-11-02,AUD,AUSTRALIAN DOLLAR,15497.32,15653.86,16156.63
3,2022-11-03,AUD,AUSTRALIAN DOLLAR,15497.32,15653.86,16156.63
4,2022-11-04,AUD,AUSTRALIAN DOLLAR,15497.32,15653.86,16156.63
...,...,...,...,...,...,...
615,2022-11-26,USD,US DOLLAR,24571.00,24601.00,24881.00
616,2022-11-27,USD,US DOLLAR,24571.00,24601.00,24881.00
617,2022-11-28,USD,US DOLLAR,24571.00,24601.00,24881.00
618,2022-11-29,USD,US DOLLAR,24571.00,24601.00,24881.00
